In [1]:
import os

In [2]:
#os.environ['PYSPARK_SUBMIT_ARGS'] = '/home/hduser/Downloads/mysql-connector-j-8.1.0.jar pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS']='--driver-class-path /usr/local/sqoop/lib/mysql-connector-java-5.1.47-bin.jar \
--jars /usr/local/sqoop/lib/mysql-connector-java-5.1.47/mysql-connector-java-5.1.47-bin.jar \
pyspark-shell'

In [3]:
import findspark
findspark.init('/usr/local/spark')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder\
    .appName('Read and write MySQL tables')\
    .master('local[*]')\
    .getOrCreate()

In [14]:
#Start all deemans and then start mysql in terminal and make sure you have created the table before executing this
#Loading the Dataframe from a table in local mysql database hari_db
#'hari_db' after 3306/ is the database name,this database contains the required tables
empDF = spark.read.format('jdbc')\
    .option('url', 'jdbc:mysql://localhost:3306/hari_db?useSSL=False')\
    .option('user', 'root')\
    .option('password', 'root')\
    .option('dbtable', 'emp')\
    .load()

In [16]:
empDF.createOrReplaceTempView('emp')

In [17]:
res_df = spark.sql("""select a.empno as emp_no,a.ename as emp_ename,b.empno as mgr_empno,b.ename as mgr_ename,
                      c.emp_cnt as mgr_tm_cnt
                      from emp a,emp b,
                      (select a.mgr,count(a.empno) as emp_cnt
                       from emp a,emp b where a.mgr=b.empno group by a.mgr) c
                       where a.mgr = b.empno and b.empno = c.mgr """)

In [19]:
#Creating the New Table emp_mgr_cnt and storing it in the local database named hari_db
res_df.write.format('jdbc')\
.mode('overwrite')\
.option('url','jdbc:mysql://localhost:3306/hari_db?useSSL=False')\
.option('user', 'root')\
.option('password', 'root')\
.option('dbtable', 'emp_mgr_cnt')\
.save()

THE END